In [1]:
openai_api_key=''

# Summarization

In [2]:
# Chat Messages

In [13]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OpenAI(streaming=True, temperature=0.7, model_name='text-davinci-003', openai_api_key=openai_api_key, callbacks=[StreamingStdOutCallbackHandler()])

# Create answer template
template="""
%INSTRUCTIONS:
Please summariza the following piese of text,.
Repsonse in a manner that a 5 years old would understand.

%TEXT:
{text}
"""

# Create a Langchain prompt template that we can insert value to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template
    )

In [14]:
story="""
From around 420 million to 350 million years ago, when land plants were still the relatively new kids on the evolutionary block and “the tallest trees stood just a few feet high,” giant spires of life poked from the Earth. “The ancient organism boasted trunks up to 24 feet (8 meters) high and as wide as three feet (one meter),” said National Geographic in 2007. With the help of a fossil dug up in Saudi Arabia, scientists finally figured out what the giant creature was: a fungus. (We think.)

The towering fungus spires would have stood out against a landscape scarce of such giants, said New Scientist in 2007.

Fossils of the organisms, known as Prototaxites, had peppered the paleontological findings of the past century and a half, ever since they were first discovered by a Canadian in 1859. But despite the fossil records, no one could figure out what the heck these giant spires were. The University of Chicago:

For the next 130 years, debate raged. Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree. “The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology. “And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”

That all changed in 2007 when a study came out that concluded the spires were a fungus, like a gigantic early mushroom.

But not everyone was sold on the idea that Prototaxites was an early fungus. No one’s questioning the spires’ existence—people just have trouble trying to imagine that such a huge structure could be a fungus. Researchers trying to refute the fungus idea thought that Prototaxites spires were gigantic mats of liverworts that had somehow rolled up. But in a follow-up study, the scientists who had proposed the fungus idea doubled down on their claim. Science is messy, and despite more than a century of digging, we still don’t really know for sure what these huge spires that dominated the ancient Earth really were.

But even though the spire-like mushrooms of yore—or whatever they were—are long gone, don’t feel too bad for funguskind. The largest organism on Earth, says ABC, is still a huge fungal mat, a single organism spread over 2,200 acres of forest in eastern Oregon.
"""

In [15]:
summarization_prompt=prompt.format(text=story)
print(summarization_prompt)


%INSTRUCTIONS:
Please summariza the following piese of text,.
Repsonse in a manner that a 5 years old would understand.

%TEXT:

From around 420 million to 350 million years ago, when land plants were still the relatively new kids on the evolutionary block and “the tallest trees stood just a few feet high,” giant spires of life poked from the Earth. “The ancient organism boasted trunks up to 24 feet (8 meters) high and as wide as three feet (one meter),” said National Geographic in 2007. With the help of a fossil dug up in Saudi Arabia, scientists finally figured out what the giant creature was: a fungus. (We think.)

The towering fungus spires would have stood out against a landscape scarce of such giants, said New Scientist in 2007.

Fossils of the organisms, known as Prototaxites, had peppered the paleontological findings of the past century and a half, ever since they were first discovered by a Canadian in 1859. But despite the fossil records, no one could figure out what the heck

In [17]:
output=llm(summarization_prompt)



A long time ago, there were giant spires growing out of the ground like huge mushrooms! They were so big that some of them were almost 25 feet tall! Scientists didn't know what they were, but after a long time, they figured out that they were probably a kind of fungus. Now, even though those big spires are gone, there's a giant fungus mat that's even bigger than them! It covers a whole forest in Oregon!

# Document Questions & Answer

In [24]:
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [25]:
loader = TextLoader('elfieco.txt')
doc = loader.load()
print(doc)

[Document(page_content="Elfie.co is a global healthcare digital company focused on revolutionising wellness solutions to enhance individual health outcomes. Our vision is to provide a holistic and gamified solution to monitor general health and chronic diseases, the silent killers of our generation.\nEach year, 25% of all people die from a chronic disease (hypertension, diabetes, cholesterol, angina, asthma…). So much of that could be avoided if people were aware of their condition and followed their doctor's prescription and lifestyle recommendations. Unfortunately, they don’t due to a lack of awareness of danger, unwillingness to change lifestyle, \xa0and costs to follow treatment (lifelong).\n\nElfie is a disruptive disease prevention and management application that gamified self-monitoring and supports patients worldwide with AI-driven coaching. It is promoted by 10,000s of doctors and endorsed by multiple healthcare leaders worldwide.\n\nIn collaboration with Willis Towers Watson,

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=200)
docs = text_splitter.split_documents(doc)

In [30]:
# Get the total number of characters.

num_chars = sum([len(x.page_content) for x in docs])
print (f"Now you have {len(docs)} documents that have an average of {num_chars / len(docs):,.0f} characters (smaller pieces)")

Now you have 38 documents that have an average of 254 characters (smaller pieces)


In [36]:
# Get embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embed documents and combine with the raw text in a pseudo db
docsearch = FAISS.from_documents(docs, embeddings)

In [39]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
query = "Who is elfie founders?"
qa.run(query)

' Elfie was founded in 2020 by Ofir Ejnes and Jean-Francois Legourd.'